In [ ]:
import easygopigo3 as easy
import numpy as np
import time 

gpg = easy.EasyGoPiGo3()
gpg_servo = gpg.init_servo('SERVO1')

my_distance_sensor = gpg.init_distance_sensor('AD2')

n=int(input("Enter the number of scans required--->"))
while n<4:
    print("Minimum value admitted is 4")
    n=int(input("Enter the number of scans required--->"))
    
dist=int(input("Enter the minimum distance at which the obstacle is detected (between 10/20 cm)--->"))
while dist<10:
    print("Not valid entry")
    dist=int(input("Enter the minimum distance at which the obstacle is detected (between 10/20 cm)--->"))

r=int(input("Enter the radius of obstacle (cm)--->"))
while r<1:
    print("Minimum value admitted is 1")
    r=int(input("Enter the radius of obstacle--->"))
    
dgr = 360/n                    #degrees of arc travelled in function of number of scans
t_sleep=1                      #to give to the sensor to move
end = False                    #stop condition when object is identified
x=1

def my_turn_degrees(angle):
    gpg.set_speed(100)
    gpg.turn_degrees(angle)
    gpg.set_speed(255)

def forward_until_obstacle():
    gpg.forward()
    while my_distance_sensor.read_mm()>120:
        pass
    gpg.stop()

def destination_free():        #prevents the gpg from crashing
    condition = my_distance_sensor.read_mm()>dist*10
    return condition
 
def surround(): 
    gpg.orbit(-dgr, dist+r)
    gpg_servo.rotate_servo(0)  #to look towards object (0 is left, circular path is positively oriented)
    time.sleep(t_sleep)

def lateral_space(a):          #check if there is enough lateral space
    gpg_servo.rotate_servo(a)
    time.sleep(t_sleep)
    free = my_distance_sensor.read_mm() > (dist+r+10)*10
    return free

def front_space(x):            #check if it is a wall or there is enough space to trace the circular path
    ang=90*(-1)**x
    my_turn_degrees(ang) 
    gpg.drive_cm(r+5) 
    if ang < 0:                #if angle is negative the gpg moves to the left so must rotate sensor to the right
        gpg_servo.rotate_servo(180) 
    else:
        gpg_servo.rotate_servo(0) 
    time.sleep(t_sleep)
    free = my_distance_sensor.read_mm() > (dist+r+10)*20 #check if circular path fits with enough space for maneuver
    return free

def identify_obstacle(x):
    my_turn_degrees(180) 
    gpg.drive_cm(r+5)          #return to point where object was detected
    my_turn_degrees(90*(-1)**x) #focus object
    gpg_servo.rotate_servo(90)
    time.sleep(t_sleep)
    scans = 0
    verify=True
    my_turn_degrees(90) #turn gpg to be aligned with circular path
    while destination_free() and scans < n and verify:
        surround()
        if (my_distance_sensor.read_mm() <= dist*10):
            scans += 1
        else:
            verify=False
        gpg_servo.rotate_servo(90)
        time.sleep(t_sleep)
    return scans == n

if __name__ == "__main__":
    gpg_servo.rotate_servo(90)
    while not end:
        forward_until_obstacle()
        a=lateral_space(0)
        b=lateral_space(180)
        if a and b:                                         #if there is lateral space on both sides...
            if front_space(x) and identify_obstacle(x):     
                print("Sentinel Identified Object")
                end = True
            else:
                ang2=90*(-1)**x
                my_turn_degrees(ang2)
                gpg_servo.rotate_servo(90)
                time.sleep(t_sleep)
        else:                                               #when no lateral space, escape through the safe path
            if a:
                my_turn_degrees(-90)
                gpg_servo.rotate_servo(90)
                time.sleep(t_sleep)
                gpg.drive_cm(30) 
                my_turn_degrees(-90)
            elif b:
                my_turn_degrees(90)
                gpg_servo.rotate_servo(90)
                time.sleep(t_sleep)
                gpg.drive_cm(30) 
                my_turn_degrees(90)
            else:                                            
                my_turn_degrees(180)
                gpg_servo.rotate_servo(90)
                time.sleep(t_sleep)
        x+=1                                                 #prevent the gpg from looping in a square so it explores the whole area